In [1]:
import pyspark
from delta import *

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/02/01 23:58:20 WARN Utils: Your hostname, DESKTOP-JJQA3IT resolves to a loopback address: 127.0.1.1; using 172.25.190.30 instead (on interface eth0)
24/02/01 23:58:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/donghee/miniconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/donghee/.ivy2/cache
The jars for the packages stored in: /home/donghee/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b16d7c45-aab6-493d-92e2-d02a3bc11e06;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 246ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

24/02/01 23:58:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data_path = '/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-create'

# Create Table

In [4]:
create_sql = "CREATE TABLE IF NOT EXISTS delta.`/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-create` ( \
	`key` STRING, \
	`value` STRING, \
	`topic` STRING, \
	`timestamp` TIMESTAMP, \
    `date` STRING \
) \
USING DELTA \
PARTITIONED BY (date) \
LOCATION '/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-create' \
TBLPROPERTIES ( \
    'delta.compatibility.symlinkFormatManifest.enabled'='true' \
)" 

In [5]:
spark_created_sql = spark.sql(create_sql)

24/02/01 23:58:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df = spark.read.format("delta").load(data_path)
df.show()

+---+-----+-----+---------+----+
|key|value|topic|timestamp|date|
+---+-----+-----+---------+----+
+---+-----+-----+---------+----+



# Basic

In [7]:
data_path = "/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-quickstart"

data = spark.range(0,5)

#### Write Data

In [8]:
data = spark.range(0,5)
data.collect()

[Row(id=0), Row(id=1), Row(id=2), Row(id=3), Row(id=4)]

In [9]:
data.write.format("delta").save(data_path)

#### Read Data

In [10]:
df = spark.read.format("delta").load(data_path)
df.show()

+---+
| id|
+---+
|  2|
|  4|
|  0|
|  3|
|  1|
+---+



#### Update Data

In [11]:
data = spark.range(30, 40)
data.write.format("delta").mode("overwrite").save(data_path)

read_df = spark.read.format("delta").load(data_path)
read_df.show()

+---+
| id|
+---+
| 37|
| 35|
| 39|
| 33|
| 36|
| 38|
| 31|
| 32|
| 34|
| 30|
+---+



#### Time Travel
- 데이터를 변경하면 자동으로 version이 생성
- 데이터를 예전 버전으로 되돌리기 가능
- 데이터의 보존 기간은 직접 수동을설정 가능
- vaccum 명령어를 사용하며 명시적으로 오래된 버전의 데이터 영구 삭제 가능

In [12]:
data_path = "/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-timetravel"

data = spark.range(0,5)
data.write.format("delta").save(data_path)

In [13]:
# History select
history = spark.sql("DESCRIBE HISTORY delta.`/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-timetravel`")
latest_version = history.selectExpr("max(version)").collect()
print("latest_version >>>>>>>>>>>>>>>: {}".format(latest_version))

latest_version >>>>>>>>>>>>>>>: [Row(max(version)=0)]


###### Current Data

In [14]:
print("##### current #####")
df = spark.read.format("delta").load(data_path)
df.show()

print("##### version 0 #####")
df0 = spark.read.format("delta").option("versionAsOf", 0).load(data_path)
df0.show()

##### current #####
+---+
| id|
+---+
|  4|
|  3|
|  2|
|  0|
|  1|
+---+

##### version 0 #####
+---+
| id|
+---+
|  4|
|  3|
|  2|
|  0|
|  1|
+---+



###### Update Data

In [15]:
data = spark.range(30, 40)
data.write.format("delta").mode("overwrite").save(data_path)

print("########### version 0 ###########")
updatedf = spark.read.format("delta").option("versionAsOf", 0).load(data_path)
updatedf.show()

print("########### current  version 1 ###########")
updatedf1 = spark.read.format("delta").option("versionAsOf", 1).load(data_path)
updatedf1.show()

########### version 0 ###########
+---+
| id|
+---+
|  4|
|  3|
|  2|
|  0|
|  1|
+---+

########### current  version 1 ###########
+---+
| id|
+---+
| 31|
| 32|
| 35|
| 34|
| 39|
| 37|
| 33|
| 38|
| 36|
| 30|
+---+



In [17]:
print("########### current ###########")
df1 = spark.read.format("delta").load(data_path)
df1.show()

history = spark.sql("DESCRIBE HISTORY delta.`/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-timetravel`")
latest_version = history.selectExpr("max(version)").collect()
print("latest_version >>>>>>>>>>>>>>>: {}".format(latest_version))

########### current ###########
+---+
| id|
+---+
| 31|
| 32|
| 35|
| 34|
| 39|
| 37|
| 33|
| 38|
| 36|
| 30|
+---+

latest_version >>>>>>>>>>>>>>>: [Row(max(version)=1)]
